## Objective
Run some naive models (both in terms of feature selection and hyperparameter tuning). This will set benchmark scores that more robustly designed models should outperform. 

In [1]:
%run data_package_loading.py # Code loads data as well as packages that are relevant across most project phases
%matplotlib inline


# !conda install -y psycopg2

from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel, RFECV 
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

# Xdb_1 = pd.read_pickle('data/madelon_db_1')
# Xdb_2 = pd.read_pickle('data/madelon_db_2')
# Xdb_3 = pd.read_pickle('data/madelon_db_3')


# ydb_1 = Xdb_1['target']
# ydb_2 = Xdb_2['target']
# ydb_3 = Xdb_3['target']
# Xdb_1 = Xdb_1.drop(['_id', 'target'], axis=1)
# Xdb_2 = Xdb_2.drop(['_id', 'target'], axis=1)
# Xdb_3 = Xdb_3.drop(['_id', 'target'], axis=1)


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def load_db():
    sql_1 = "SELECT * FROM madelon TABLESAMPLE BERNOULLI (1) REPEATABLE (42*1)"
    sql_2 = "SELECT * FROM madelon TABLESAMPLE BERNOULLI (1) REPEATABLE (42*2)"
    sql_3 = "SELECT * FROM madelon TABLESAMPLE BERNOULLI (1) REPEATABLE (42*3)"

    df_db_1 = madelon_db_sql(sql_1)
    sleep(1)
    df_db_2 = madelon_db_sql(sql_2)
    sleep(1)
    df_db_3 = madelon_db_sql(sql_3)
    df_db_1 = pd.DataFrame(df_db_1)
    df_db_2 = pd.DataFrame(df_db_2)
    df_db_3 = pd.DataFrame(df_db_3)
    
    Xdb_1 = df_db_1.drop(['_id','target'], axis=1)
    Xdb_2 = df_db_2.drop(['_id','target'], axis=1)
    Xdb_3 = df_db_3.drop(['_id','target'], axis=1)
    ydb_1 = df_db_1['target']
    ydb_2 = df_db_2['target']
    ydb_3 = df_db_3['target']
    
    return Xdb_1, Xdb_2, Xdb_3, ydb_1, ydb_2, ydb_3

Xdb_1, Xdb_2, Xdb_3, ydb_1, ydb_2, ydb_3 = load_db()

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

In [4]:
data_target = [(Xuci_1, yuci_1, 'uci_1'), 
               (Xuci_2, yuci_2, 'uci_2'), 
               (Xuci_3, yuci_3, 'uci_3'), 
               (Xdb_1,  ydb_1, 'db_1'), 
               (Xdb_2,  ydb_2, 'db_2'), 
               (Xdb_3,  ydb_3, 'db_3')]

In [5]:
C = 10 ** 9
model_list = [LogisticRegression(C = C), 
              DecisionTreeClassifier(), 
              KNeighborsClassifier(n_jobs=-1), 
              SVC(C = C)]

In [6]:
def run_pipe_cv (X, y, data_name, model):
    
#     # Test Train split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    # Set up Pipeline
    pipe = Pipeline([
                  ('scaler', StandardScaler()),
                  ('model', model)
                  ])
    
    # Set up Kfolds split
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
    skf.get_n_splits(X, y)
    
    # Run pipe on Kfold
    train_scores = []
    test_scores = []
    for train_cv_index, val_cv_index in skf.split(X, y):
        X_train_temp = X.iloc[train_cv_index, :]
        y_train_temp = y[train_cv_index]
        X_test_temp = X.iloc[val_cv_index, :]
        y_test_temp = y[val_cv_index]
        
        pipe.fit(X_train_temp, y_train_temp)
        train_scores.append(pipe.score(X_train_temp, y_train_temp))
        test_scores.append(pipe.score(X_test_temp, y_test_temp))

    results.append({
                'test_train': 'train',
                'data': data_name,
                'scaler': pipe.named_steps['scaler'],
                'model': pipe.named_steps['model'],
                'score': np.mean(train_scores) #accuracy for classifications?
               })
    results.append({
            'test_train': 'test',
            'data': data_name,
            'scaler': pipe.named_steps['scaler'],
            'model': pipe.named_steps['model'],
            'score': np.mean(test_scores) #accuracy for classifications?
           })
        
    # How do I run the model on the test data?

In [7]:
results = []

for Xy in data_target:
    for model in tqdm(model_list):
        run_pipe_cv(Xy[0], Xy[1], Xy[2], model)
        
results_df = pd.DataFrame(results)

100%|██████████| 4/4 [02:41<00:00, 40.45s/it]


In [8]:
# training scores
results_df[results_df['test_train'] == 'train'].sort_values('score', ascending=False)

,data,model,scaler,score,test_train
0,uci_1,"LogisticRegression(C=1000000000, class_weight=...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
22,uci_3,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
42,db_3,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
40,db_3,"LogisticRegression(C=1000000000, class_weight=...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
38,db_2,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
34,db_2,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
32,db_2,"LogisticRegression(C=1000000000, class_weight=...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
30,db_1,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
26,db_1,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train
2,uci_1,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",1.000000,train


In [9]:
# training scores
results_df[results_df['test_train'] == 'test'].sort_values('score', ascending=False)

,data,model,scaler,score,test_train
27,db_1,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",0.612137,test
35,db_2,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",0.600156,test
11,uci_2,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",0.597727,test
43,db_3,"DecisionTreeClassifier(class_weight=None, crit...","StandardScaler(copy=True, with_mean=True, with...",0.597360,test
15,uci_2,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",0.584091,test
23,uci_3,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",0.581951,test
47,db_3,"SVC(C=1000000000, cache_size=200, class_weight...","StandardScaler(copy=True, with_mean=True, with...",0.562589,test
9,uci_2,"LogisticRegression(C=1000000000, class_weight=...","StandardScaler(copy=True, with_mean=True, with...",0.552273,test
13,uci_2,"KNeighborsClassifier(algorithm='auto', leaf_si...","StandardScaler(copy=True, with_mean=True, with...",0.552273,test
37,db_2,"KNeighborsClassifier(algorithm='auto', leaf_si...","StandardScaler(copy=True, with_mean=True, with...",0.549934,test


In [11]:
results_df.to_csv('data/benchmark_results_df.csv', index=False)